# Cleaning the Raw Data

## This Notebook is to clean the parks and crime data. Both crime datasets will be cleansed simultaneously.

In [1]:
import os
import pandas as pd
import re

#### Cleaning the parks file (1 of 3)

In [2]:
parks= os.path.join("../Resources/Raw/city-of-austin-parks-1.csv")

In [3]:
parks_df= pd.read_csv(parks)

parks_df = parks_df[["PARK_ID","PARK_ACRES","SHAPE_AREA","SHAPE_LEN","ZIP_CODE"]]
parks_df.head()

,PARK_ID,PARK_ACRES,SHAPE_AREA,SHAPE_LEN,ZIP_CODE
0,215,1.152581,5.020624e+04,912.073535,78731
1,313,4.943708,2.153470e+05,2456.140724,78703
2,371,77.390907,3.371134e+06,9262.547633,78746
3,289,2.248317,9.793630e+04,1665.906556,78703
4,315,91.228997,3.973919e+06,35791.080093,78745


In [4]:
zip_parks_df = parks_df.groupby(["ZIP_CODE"])["PARK_ID"].count().reset_index()
zip_parks_df= zip_parks_df.rename(columns={"ZIP_CODE": "Zip Code","PARK_ID": "Number of Parks"})

zip_parks_df.head()

,Zip Code,Number of Parks
0,78613,1
1,78617,5
2,78652,2
3,78660,1
4,78664,1


In [5]:
zip_parks_df.to_csv("../Resources/CLean/parks_by_zip.csv")

#### Cleaning Crime Data

In [6]:
crime_2014= os.path.join("../Resources/Raw/Annual_Crime_2014.csv")
crime_2016= os.path.join("../Resources/Raw/2016_Annual_Crime_Data.csv.")

In [7]:
crime_2014_df = pd.read_csv(crime_2014)
crime_2016_df = pd.read_csv(crime_2016)

In [8]:
crime_2014_df = crime_2014_df[["GO Location Zip","GO Highest Offense Desc","Highest NIBRS/UCR Offense Description"]]
crime_2014_df= crime_2014_df.rename(columns={"GO Location Zip": "Zip Code", "Highest NIBRS/UCR Offense Description":"UCR Description"})

In [9]:
# Decided to remove the zip codes with less than 40 offenses

# crime_2014_df["Zip Code"].value_counts()
outliers= ["78,653", "78,712","78,728","78,652","78,732","78,733","78,610","78,737"]
for outlier in outliers:
    crime_2014_df = crime_2014_df[crime_2014_df["Zip Code"] != outlier]
crime_2014_df["Zip Code"].value_counts()

78,753    3935
78,741    3394
78,758    2826
78,704    2715
78,745    2577
78,702    2196
78,744    2195
78,723    2083
78,701    2036
78,759    1557
78,748    1550
78,752    1321
78,705    1138
78,757    1098
78,751     926
78,749     818
78,746     790
78,703     660
78,721     639
78,724     537
78,727     532
78,731     527
78,729     469
78,754     457
78,750     409
78,613     404
78,756     355
78,722     347
78,735     280
78,617     274
78,717     257
78,747     249
78,726     204
78,660     146
78,739     133
78,719     127
78,725      76
78,736      57
78,730      51
78,742      46
Name: Zip Code, dtype: int64

In [10]:
crime_2014_df["UCR Description"].unique()

array(['Robbery', 'Burglary / \nBreaking & Entering', 'Auto Theft',
       'Homicide: Murder & Nonnegligent Manslaughter', 'Rape',
       'Aggravated Assault', 'Theft: Shoplifting',
       'Theft: Pocket Picking', 'Theft: Purse Snatching',
       'Theft: from Building', 'Theft: Coin Op Machine', 'Theft: BOV',
       'Theft: Auto Parts', 'Theft: All Other Larceny'], dtype=object)

In [12]:
crime_2016_df= crime_2016_df[["GO Location Zip","GO Highest Offense Desc","Highest NIBRS/UCR Offense Description"]]
crime_2016_df= crime_2016_df.rename(columns={"GO Location Zip": "Zip Code", "Highest NIBRS/UCR Offense Description":"UCR Description"})

In [27]:
# Decided to remove the zip codes with less than 40 offenses

# crime_2016_df["Zip Code"].value_counts()

outliers = [78652.0,78712.0,78728.0,78664.0,78732.0,78737.0]

for outlier in outliers:
    crime_2016_df = crime_2016_df[crime_2016_df["Zip Code"] != outlier]
crime_2016_df["Zip Code"].value_counts()

78741.0    3355
78753.0    3134
78704.0    2557
78758.0    2499
78745.0    2245
78701.0    2076
78723.0    2041
78744.0    1990
78702.0    1582
78748.0    1372
78752.0    1289
78759.0    1198
78705.0    1134
78757.0    1074
78751.0     914
78746.0     826
78749.0     790
78703.0     660
78729.0     564
78724.0     540
78727.0     538
78721.0     515
78731.0     512
78754.0     413
78613.0     341
78750.0     340
78756.0     326
78717.0     305
78735.0     300
78617.0     285
78722.0     261
78747.0     229
78726.0     221
78660.0     179
78719.0     165
78739.0     151
78736.0     115
78725.0      72
78742.0      58
78653.0      48
78730.0      46
Name: Zip Code, dtype: int64

In [14]:
crime_2016_df["UCR Description"].unique()

array(['Agg Assault', 'Theft', 'Robbery', 'Rape', 'Burglary',
       'Auto Theft', 'Murder'], dtype=object)

#### Write a function that will make both 2016 & 2014's Crime Type columns to match

In [15]:
def cleanNIBRS(offense):
    offenses = ['Agg Assault','Auto Theft', 'Robbery', 'Rape', 'Burglary', 'Theft', 'Murder']
    for off in offenses:
        if re.search(off, offense):
            return off
        elif offense == "Aggravated Assault":
            return 'Agg Assault'
        else:
            continue

In [16]:
crime_2014_df["UCR Description"] = crime_2014_df["UCR Description"].apply(lambda row: cleanNIBRS(row))

In [17]:
crime_2014_df["UCR Description"].unique()

array(['Robbery', 'Burglary', 'Auto Theft', 'Murder', 'Rape',
       'Agg Assault', 'Theft'], dtype=object)

In [18]:
crime_2014_df.head()

,Zip Code,GO Highest Offense Desc,UCR Description
0,"78,753",AGG ROBBERY/DEADLY WEAPON,Robbery
1,"78,723",ROBBERY BY ASSAULT,Robbery
2,"78,702",ROBBERY BY THREAT,Robbery
3,"78,723",AGG ROBBERY/DEADLY WEAPON,Robbery
4,"78,702",AGG ROBBERY/DEADLY WEAPON,Robbery


In [19]:
crime_2016_df.head()

,Zip Code,GO Highest Offense Desc,UCR Description
0,78735.0,AGG ASLT ENHANC STRANGL/SUFFOC,Agg Assault
1,78701.0,THEFT,Theft
2,78753.0,AGG ROBBERY/DEADLY WEAPON,Robbery
3,78701.0,THEFT,Theft
4,78753.0,SEXUAL ASSAULT W/ OBJECT,Rape


In [20]:
crime_2016_df.to_csv("../Resources/CLean/crime_2016_clean.csv")
crime_2014_df.to_csv("../Resources/CLean/crime_2014_clean.csv")